In [3]:
import statsmodels.api as sm
import numpy as np
import LinearModel as lm

size = 100
np.random.seed(1)
X1 = np.random.normal(loc=0, scale=1, size=size)       
X2 = np.random.normal(loc=0, scale=1, size=size)   
X3 = np.random.normal(loc=0, scale=1, size=size) 
X = np.concatenate([X1, X2, X3]).reshape(size, 3)
y = 1 + 2*X[:,0] + -3*X[:,1] + 0*X[:,2] + np.random.normal(loc=0, scale=1, 
                                                          size=size)      
        


# Create an array of 1s equal in length to the observations in X.
intercept_column = np.repeat(1, repeats=X.shape[0])
# Insert it at the 0-th column index.
X_copy = np.insert(X, 0, intercept_column, axis=1)
        
        
sm_model = sm.OLS(y, X_copy).fit()
print(sm_model.summary())       
        

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     396.9
Date:                Sun, 14 Feb 2021   Prob (F-statistic):           5.94e-54
Time:                        02:49:58   Log-Likelihood:                -147.38
No. Observations:                 100   AIC:                             302.8
Df Residuals:                      96   BIC:                             313.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9915      0.109      9.067      0.0

In [4]:
my_ols = lm.OLS()
my_ols.fit(X, y, method="qr")  

print(my_ols.df_model)
print(my_ols.df_residuals)
print(my_ols.R_sq)
print(my_ols.adj_R_sq)
print(my_ols.F_stat)
print(my_ols.F_prob)
print(my_ols.beta_hat)
print(my_ols.beta_hat_se)
print(my_ols.beta_hat_t_stats)
print(my_ols.beta_hat_prob)

3
96
0.9253906152817284
0.9230590720092824
396.9004676936213
1.1102230246251565e-16
[ 0.99149103  2.02639271 -3.1349261  -0.0052813 ]
[0.10935505 0.10864965 0.1171335  0.12014304]
[  9.06671499  18.65070685 -26.7637022   -0.04395841]
[1.50990331e-14 0.00000000e+00 0.00000000e+00 9.65028860e-01]


In [5]:
my_ols = lm.OLS()
my_ols.fit(X, y, method="moore-penrose")  

print(my_ols.df_model)
print(my_ols.df_residuals)
print(my_ols.R_sq)
print(my_ols.adj_R_sq)
print(my_ols.F_stat)
print(my_ols.F_prob)
print(my_ols.beta_hat)
print(my_ols.beta_hat_se)
print(my_ols.beta_hat_t_stats)
print(my_ols.beta_hat_prob)

3
96
0.9253906152817284
0.9230590720092824
396.9004676936213
1.1102230246251565e-16
[ 0.99149103  2.02639271 -3.1349261  -0.0052813 ]
[0.10935505 0.10864965 0.1171335  0.12014304]
[  9.06671499  18.65070685 -26.7637022   -0.04395841]
[1.50990331e-14 0.00000000e+00 0.00000000e+00 9.65028860e-01]


In [6]:
my_ols = lm.OLS()
my_ols.fit(X, y, method="svd")  

print(my_ols.df_model)
print(my_ols.df_residuals)
print(my_ols.R_sq)
print(my_ols.adj_R_sq)
print(my_ols.F_stat)
print(my_ols.F_prob)
print(my_ols.beta_hat)
print(my_ols.beta_hat_se)
print(my_ols.beta_hat_t_stats)
print(my_ols.beta_hat_prob)

3
96
0.9253906152817284
0.9230590720092824
396.9004676936211
1.1102230246251565e-16
[ 0.99149103  2.02639271 -3.1349261  -0.0052813 ]
[0.10935505 0.10864965 0.1171335  0.12014304]
[  9.06671499  18.65070685 -26.7637022   -0.04395841]
[1.50990331e-14 0.00000000e+00 0.00000000e+00 9.65028860e-01]
